# Part 2 (25 points)

## Write a function that estimates the emission parameters from the training set using MLE

In [1]:
def load_dataset(file_path):
    """
    For loading a dataset.
    
    Parameters:
    file_path (string): File path of the dataset
    
    Returns:
    list: List of strings of each line in the dataset
    
    """
    with open(file_path) as file:
        data = file.readlines()
    file.close()
    
    return data

# TODO: load your training dataset

al = load_dataset('dataset/al/AL/train')
cn = load_dataset('dataset/cn/CN/train')
en = load_dataset('dataset/en/EN/train')
sg = load_dataset('dataset/sg/SG/train')

In [2]:
def get_emission_parameters(train_data, k=3, token='#UNK#'):
    """
    For getting emission parameters for a training dataset.
    
    Parameters:
    train_data (list): Data loaded from your training data with the "load_dataset" function
    k (int): Word frequency threshold (default 3)
    token (string): Token representing words that appear less than the frequency threshold (default #UNK#)
    
    Returns:
    dict: Dictionary of emission parameters, with format {word:{tag:count}}
    
    """
    assert type(k) == int
    assert train_data != None
    
    emission_dict = {}
    x_count = {}
    y_count = {}
    
    # get word count for each word in the training dataset
    for d in train_data:
        d = d.strip('\n').split(' ')
        try:
            x = d[0]
            y = d[1]
        except:
            continue
        if x not in x_count.keys():
            x_count[x] = 1
        else:
            x_count[x] += 1
        if y not in y_count.keys():
            y_count[y] = 1
        else:
            y_count[y] += 1
            
    # process data
    for d in train_data:
        d = d.strip('\n').split(' ')
        try:
            x = d[0]
            y = d[1]
        except:
            continue
        # add emission paramters for word if word appears more than frequency k
        if x_count[x] >= k:
            if x not in emission_dict.keys():
                emission_dict[x] = {y:1}
            elif y not in emission_dict[x].keys():
                emission_dict[x][y] = 1
            else:
                emission_dict[x][y] += 1
        # change word to unknown token and add emission paramters if word does not appear more than frequency k
        elif x_count[x] < k:
            if token not in emission_dict.keys():
                emission_dict[token] = {y:1}
            elif y not in emission_dict[token].keys():
                emission_dict[token][y] = 1
            else:
                emission_dict[token][y] += 1
            
    # maximum likelihood estimation for emission parameters
    for i in emission_dict.keys():
        for k in emission_dict[i].keys():
            emission_dict[i][k] /= y_count[k]
    
    return emission_dict


# TODO: get the emission parameters for your training dataset

al_emission = get_emission_parameters(al)
cn_emission = get_emission_parameters(cn)
en_emission = get_emission_parameters(en)
sg_emission = get_emission_parameters(sg)

In [3]:
import os
import operator

def sentiment_tag_output(emission_dict, test_file_path, output_path, token='#UNK#'):
    """
    For tagging a test dataset with emission parameters from a train dataset.
    
    Parameters:
    emission_dict (dict): Dictionary of emission parameters, with format {word:{tag:count}}
    test_file_path (string): File path of the test dataset
    output_path (string): File path of the output sequences with predicted tags
    token (string): Token representing unknown words in the test dataset (default #UNK#)
    
    """
    assert emission_dict != None
    assert os.path.isfile(test_file_path)
    assert output_path != None
    
    test_data = load_dataset(test_file_path)
    
    keys = emission_dict.keys()
    
    for i in range(len(test_data)):
        test_data[i] = test_data[i].strip('\n')
        x = test_data[i]

        if x != '':
            # check if test dataset token appears in trained emission parameters, if not replace it with the unknown token
            if x not in keys:
                x = token
            arg_max_y = max(emission_dict[x].items(), key=operator.itemgetter(1))[0]
            
            test_data[i] += ' ' + arg_max_y + '\n'
        else:
            test_data[i] += '\n'
            continue
    
    # write outputs to output_path
    with open(output_path, 'w') as f:
        for item in test_data:
            f.write(item)
        f.close()

        
# TODO: get the naive tagger output for your test dataset

sentiment_tag_output(al_emission, 'dataset/al/AL/dev.in', 'dataset/al/AL/dev.p2.out')
sentiment_tag_output(cn_emission, 'dataset/cn/CN/dev.in', 'dataset/cn/CN/dev.p2.out')
sentiment_tag_output(en_emission, 'dataset/en/EN/dev.in', 'dataset/en/EN/dev.p2.out')
sentiment_tag_output(sg_emission, 'dataset/sg/SG/dev.in', 'dataset/sg/SG/dev.p2.out')

In [4]:
# Evaluation results for Part 2

print("\n\n**EN Development Set**")
!python3 evalscript/evalResult.py dataset/en/EN/dev.out dataset/en/EN/dev.p2.out
print("\n\n**AL Development Set**")
!python3 evalscript/evalResult.py dataset/al/AL/dev.out dataset/al/AL/dev.p2.out
print("\n\n**CN Development Set**")
!python3 evalscript/evalResult.py dataset/cn/CN/dev.out dataset/cn/CN/dev.p2.out
print("\n\n**SG Development Set**")
!python3 evalscript/evalResult.py dataset/sg/SG/dev.out dataset/sg/SG/dev.p2.out



**EN Development Set**

#Entity in gold data: 13179
#Entity in prediction: 19406

#Correct Entity : 9152
Entity  precision: 0.4716
Entity  recall: 0.6944
Entity  F: 0.5617

#Correct Sentiment : 7644
Sentiment  precision: 0.3939
Sentiment  recall: 0.5800
Sentiment  F: 0.4692


**AL Development Set**

#Entity in gold data: 8408
#Entity in prediction: 19484

#Correct Entity : 2898
Entity  precision: 0.1487
Entity  recall: 0.3447
Entity  F: 0.2078

#Correct Sentiment : 2457
Sentiment  precision: 0.1261
Sentiment  recall: 0.2922
Sentiment  F: 0.1762


**CN Development Set**

#Entity in gold data: 1478
#Entity in prediction: 9373

#Correct Entity : 765
Entity  precision: 0.0816
Entity  recall: 0.5176
Entity  F: 0.1410

#Correct Sentiment : 285
Sentiment  precision: 0.0304
Sentiment  recall: 0.1928
Sentiment  F: 0.0525


**SG Development Set**

#Entity in gold data: 4537
#Entity in prediction: 18451

#Correct Entity : 2632
Entity  precision: 0.1426
Entity  recall: 0.5801
Entity  F: 0.2290



# Part 3 (20 points)

In [5]:
def get_transition_parameters(train_data, start='START', stop='STOP'):
    """
    For getting transition parameters for a training dataset.
    
    Parameters:
    train_data (dict): Data loaded from your training data with the "load_dataset" function
    start (string): Start token representation (default START)
    stop (string): Stop token representation (default STOP)
    
    Returns:
    dict: Dictionary of transition parameters, with format {current_state:{next_state:count}}
    
    """
    assert train_data != None
    
    transition_dict = {}
    y_count = {}
    
    # account for first missing start
    y_count[start] = 1
    # account for last missing stop
    y_count[stop] = 1
    
    # get tag count for each tag in the training dataset
    for d in train_data:
        d = d.strip('\n').split(' ')
        try:
            y = d[1]
        except:
            y_count[start] += 1
            y_count[stop] += 1
            continue
        if y not in y_count.keys():
            y_count[y] = 1
        else:
            y_count[y] += 1
            
    # process data to get transition parameters
    for i in range(len(train_data)):
        try:
            y = train_data[i].strip('\n').split(' ')[1]
        except:
            continue
            
        try:
            by = train_data[i-1].strip('\n').split(' ')[1]
        except:
            by = start
            
        try:
            fy = train_data[i+1].strip('\n').split(' ')[1]
        except:
            fy = stop
        
        if by == start:
            if by not in transition_dict.keys():
                transition_dict[by] = {y:1}
            elif y not in transition_dict[by].keys():
                transition_dict[by][y] = 1
            else:
                transition_dict[by][y] += 1
        
        if y not in transition_dict.keys():
            transition_dict[y] = {fy:1}
        elif fy not in transition_dict[y].keys():
            transition_dict[y][fy] = 1
        else:
            transition_dict[y][fy] += 1
            
    # maximum likelihood estimation for transition parameters
    for i in transition_dict.keys():
        for k in transition_dict[i].keys():
            transition_dict[i][k] /= y_count[i]
    
    return transition_dict


# TODO: get the transition parameters for your training dataset

al_transition = get_transition_parameters(al)
cn_transition = get_transition_parameters(cn)
en_transition = get_transition_parameters(en)
sg_transition = get_transition_parameters(sg)

In [6]:
import numpy as np
import random

def get_viterbi_output(emission_dict, transition_dict, test_file_path, output_path, nan_layer_tag='O', best_n=1):
    """
    For tagging test dataset with the Viterbi algorithm.
    
    emission_dict (dict): Dictionary of emission parameters, with format {word:{tag:count}}
    transition_dict (dict): Dictionary of transition parameters, with format {current_state:{next_state:count}}
    test_file_path (string): File path of the test dataset
    output_path (string): File path of the output sequences with predicted tags
    nan_layer_tag (string): Token for layers with no nodes with non-NaN values
    best_n (int): Choice for getting best_n sequences
    
    """    
    assert transition_dict != None
    assert best_n != None
    assert os.path.isfile(test_file_path)
    assert output_path != None
    
    test_data = load_dataset(test_file_path)
    sequences = []
    temp_seq = []
    output = []
    
    # load test data
    for i in range(len(test_data)):
        test_data[i] = test_data[i].strip('\n')
        if test_data[i] != '':
            temp_seq.append(test_data[i])
        else:
            sequences.append(temp_seq)
            temp_seq = []
            
    # get set of all states
    states = list(set([x for x in transition_dict.keys() if x != 'START'])) # correspond to columns, index 0 = first row
    
    for seq in sequences:
        # recursive forward pass
        length = len(seq) # not including START and STOP
        scores = np.zeros((length, len(states)))

        # values converted to log to prevent numerical underflow
        for row in range(scores.shape[0]):
            for col in range(scores.shape[1]):
                if row == 0:
                    try:
                        if seq[row] in emission_dict.keys():
                            scores[row,col] = 1 + np.log(transition_dict['START'][states[col]]) + np.log(emission_dict[seq[row]][states[col]])
                        else:
                            scores[row,col] = 1 + np.log(transition_dict['START'][states[col]]) + np.log(emission_dict['#UNK#'][states[col]])
                    except:
                        scores[row,col] = np.nan
                else:
                    state_score = []
                    for prev_state in range(len(states)):
                        try:
                            if seq[row] in emission_dict.keys():
                                score = scores[row-1,prev_state] + np.log(transition_dict[states[prev_state]][states[col]]) + np.log(emission_dict[seq[row]][states[col]])
                            else:
                                score = scores[row-1,prev_state] + np.log(transition_dict[states[prev_state]][states[col]]) + np.log(emission_dict['#UNK#'][states[col]])
                        except:
                            score = np.nan
                        state_score.append(score)
                    scores[row,col] = np.nanmax(state_score)
                            
            min_score = np.nanmin(scores[row,:])
            
            # post-process to make sure no zeros and no negatives for next row's log operations by adding the absolute value of the smallest node value with a small constant
            for col in range(len(states)):
                if np.isnan(scores[row,col]):
                    pass
                else:
                    scores[row,col] += np.abs(min_score) + 1e-5
        
        # search for the best_n path's last node
        tags = []
        left = best_n
        ignored_layers = 0
        
        for row in range(length-1,-1,-1):
            STOP = np.zeros(len(states))

            for col in range(len(states)):
                try:
                    STOP[col] = scores[row,col] + np.log(transition_dict[states[col]]['STOP'])
                except:
                    STOP[col] = np.nan
                    
            # check the number of non-nan values in the layer
            num_valid_values = STOP.size - np.isnan(STOP).sum()
            
            if left - num_valid_values <= 0:
                # get best_n node
                index_choice = np.argsort(STOP)[-left-np.isnan(STOP).sum()]
                tags.append(states[index_choice])
                ignored_layers += 1
                break
            else:
                # if last layer(s) do not have sufficient nodes with non-NaN scores for the best_n sequence, we continue looking
                left -= num_valid_values
                ignored_layers += 1
                tags.append(nan_layer_tag)
        
        # backward pass for best_n path
        if ignored_layers > 1:
            # get the best tags for the case where the last valid layer is not the last layer before STOP
            backward_score = np.asarray([])
            
            # get the best tag for the last valid layer if it is not the last layer before STOP
            for col in range(len(states)):
                # only use the emission parameters for last valid layer if it is not the last layer before STOP
                # experimentally, this provides better accuracy than considering the transition parameters to a fake STOP
                try:
                    backward_score = np.append(backward_score, scores[length-1-ignored_layers,col])
                except:
                    backward_score = np.append(backward_score, np.nan)      
            try:
                index_choice = np.argsort(backward_score)[-1-np.isnan(backward_score).sum()]
                tags.append(states[index_choice])
            except:
                tags.append(nan_layer_tag)
                                  
            # get the best tag for the previous layers
            for row in range(length-2-ignored_layers,-1,-1):
                backward_score = np.asarray([])
                for col in range(len(states)):
                    try:
                        backward_score = np.append(backward_score, scores[row,col] * transition_dict[states[col]][tags[-1]])
                    except:
                        backward_score = np.append(backward_score, np.nan)      
                try:
                    index_choice = np.argsort(backward_score)[-1-np.isnan(backward_score).sum()]
                    tags.append(states[index_choice])
                except:
                    tags.append(nan_layer_tag)
        else:
            # get the best tags for the case where the last valid layer is the last layer before STOP
            for row in range(length-2,-1,-1):
                backward_score = np.asarray([])
                for col in range(len(states)):
                    try:
                        backward_score = np.append(backward_score, scores[row,col] * transition_dict[states[col]][tags[-1]])
                    except:
                        backward_score = np.append(backward_score, np.nan)      
                try:
                    index_choice = np.argsort(backward_score)[-1-np.isnan(backward_score).sum()]
                    tags.append(states[index_choice])
                except:
                    tags.append(nan_layer_tag)
            
        # reverse tag outputs
        tags.reverse()
        
        for i in range(len(seq)):
            output.append(seq[i] + ' ' + tags[i] + '\n')
        output.append('\n')
        
    # write outputs to output_path
    with open(output_path, 'w') as f:
        for item in output:
            f.write(item)
        f.close()

        
# TODO: get the Viterbi algorithm's best outputs for your test dataset
        
get_viterbi_output(al_emission, al_transition, 'dataset/al/AL/dev.in', 'dataset/al/AL/dev.p3.out', nan_layer_tag='B-REDUNDANT')
get_viterbi_output(cn_emission, cn_transition, 'dataset/cn/CN/dev.in', 'dataset/cn/CN/dev.p3.out')
get_viterbi_output(en_emission, en_transition, 'dataset/en/EN/dev.in', 'dataset/en/EN/dev.p3.out')
get_viterbi_output(sg_emission, sg_transition, 'dataset/sg/SG/dev.in', 'dataset/sg/SG/dev.p3.out')

/home/samson/.local/lib/python3.6/site-packages/ipykernel_launcher.py:65: RuntimeWarning: All-NaN axis encountered
/home/samson/.local/lib/python3.6/site-packages/ipykernel_launcher.py:67: RuntimeWarning: All-NaN slice encountered


In [7]:
# Evaluation results for Part 3

print("\n\n**EN Development Set**")
!python3 evalscript/evalResult.py dataset/en/EN/dev.out dataset/en/EN/dev.p3.out
print("\n\n**AL Development Set**")
!python3 evalscript/evalResult.py dataset/al/AL/dev.out dataset/al/AL/dev.p3.out
print("\n\n**CN Development Set**")
!python3 evalscript/evalResult.py dataset/cn/CN/dev.out dataset/cn/CN/dev.p3.out
print("\n\n**SG Development Set**")
!python3 evalscript/evalResult.py dataset/sg/SG/dev.out dataset/sg/SG/dev.p3.out



**EN Development Set**

#Entity in gold data: 13179
#Entity in prediction: 14106

#Correct Entity : 9584
Entity  precision: 0.6794
Entity  recall: 0.7272
Entity  F: 0.7025

#Correct Sentiment : 8700
Sentiment  precision: 0.6168
Sentiment  recall: 0.6601
Sentiment  F: 0.6377


**AL Development Set**

#Entity in gold data: 8408
#Entity in prediction: 10372

#Correct Entity : 4940
Entity  precision: 0.4763
Entity  recall: 0.5875
Entity  F: 0.5261

#Correct Sentiment : 3320
Sentiment  precision: 0.3201
Sentiment  recall: 0.3949
Sentiment  F: 0.3536


**CN Development Set**

#Entity in gold data: 1478
#Entity in prediction: 707

#Correct Entity : 294
Entity  precision: 0.4158
Entity  recall: 0.1989
Entity  F: 0.2691

#Correct Sentiment : 200
Sentiment  precision: 0.2829
Sentiment  recall: 0.1353
Sentiment  F: 0.1831


**SG Development Set**

#Entity in gold data: 4537
#Entity in prediction: 2682

#Correct Entity : 1586
Entity  precision: 0.5913
Entity  recall: 0.3496
Entity  F: 0.4394

#C

# Part 4 (20 points)

In [8]:
# TODO: get the Viterbi algorithm's 7th best outputs for your test dataset

get_viterbi_output(al_emission, al_transition, 'dataset/al/AL/dev.in', 'dataset/al/AL/dev.p4.out', best_n=7, nan_layer_tag='B-REDUNDANT')
get_viterbi_output(en_emission, en_transition, 'dataset/en/EN/dev.in', 'dataset/en/EN/dev.p4.out', best_n=7)

# Evaluation results for Part 4

print("\n\n**EN Development Set**")
!python3 evalscript/evalResult.py dataset/en/EN/dev.out dataset/en/EN/dev.p4.out
print("\n\n**AL Development Set**")
!python3 evalscript/evalResult.py dataset/al/AL/dev.out dataset/al/AL/dev.p4.out

/home/samson/.local/lib/python3.6/site-packages/ipykernel_launcher.py:65: RuntimeWarning: All-NaN axis encountered
/home/samson/.local/lib/python3.6/site-packages/ipykernel_launcher.py:67: RuntimeWarning: All-NaN slice encountered




**EN Development Set**

#Entity in gold data: 13179
#Entity in prediction: 13711

#Correct Entity : 8826
Entity  precision: 0.6437
Entity  recall: 0.6697
Entity  F: 0.6565

#Correct Sentiment : 7959
Sentiment  precision: 0.5805
Sentiment  recall: 0.6039
Sentiment  F: 0.5920


**AL Development Set**

#Entity in gold data: 8408
#Entity in prediction: 11380

#Correct Entity : 4108
Entity  precision: 0.3610
Entity  recall: 0.4886
Entity  F: 0.4152

#Correct Sentiment : 2113
Sentiment  precision: 0.1857
Sentiment  recall: 0.2513
Sentiment  F: 0.2136


# Part 5 – Design Challenge (20 points)

In [36]:
import numpy as np
from collections import defaultdict
import random
    
class AveragedPerceptronTagger:
    def __init__(self):
        """
        Initialisation of necessary components for the averaged perceptron tagger.
        """
        # records each timestep and timestep each feature is last updated
        self.time = 0
        self.update_time = defaultdict(lambda: defaultdict(int))
        # stores the tag that are most likely to be emitted by each word, depending on the frequency and ambiguity thresholds
        self.tagdict = {}
        # set of all tags
        self.tags = set()
        # stores weights, in {feature:{tag:weight} format
        self.weights = {}
        # cumulative total of the weights
        self.total_weights = defaultdict(lambda: defaultdict(int))
        
        
    def get_features(self, i, word, sequence, past_tags, language):
        """
        Feature templates for creating features.
        
        Parameters:
        i (int): Timestep of the sentence being considered
        word (string): Word in the sentence being considered
        sequence (list): The sequence being considered
        past_tags (list): List of the tags already predicted
        language (string): Language to determine choice of feature templates
        
        Returns:
        dict: Dictionary of the feature representation of a word/token in a sequence
        
        """
        def add_feature(name, *args):
            """
            Add feature to feature dictionary.
            
            Parameters:
            *args (string): String representations of values to be added to feature representation
            
            """
            features[' '.join((name,) + tuple(args))] += 1

        features = defaultdict(int)
        
        if language == "english":
            add_feature('bias')
            add_feature('word_length', str(len(word)))
            add_feature('is_capitalized', str(word[0].upper() == word[0]))
            add_feature('is_all_capitalized', str(word.upper() == word))
            add_feature('is_capitals_inside', str(word[1:].lower() != word[1:]))
            add_feature('is_numeric', str(word.isdigit()))
            add_feature('contains_num', str(any(char.isdigit() for char in word)))
            add_feature('3-prev-tag', '' if i < 3 else past_tags[i-3])
            add_feature('2-prev-tag', '' if i < 2 else past_tags[i-2])
            add_feature('prev-tag', '' if i < 1 else past_tags[i-1])
            add_feature('suffix-1', word[-1:])
            add_feature('suffix-2', '' if len(word) < 2 else word[-2:])
            add_feature('suffix-3', '' if len(word) < 3 else word[-3:])
            add_feature('prefix-1', word[0])
            add_feature('prefix-2', '' if len(word) < 2 else word[:2])
            add_feature('prefix-3', '' if len(word) < 3 else word[:3])
            add_feature('word', word)
            add_feature('prev-word', '' if i < 1 else sequence[i-1])
            add_feature('prev-word-with-tag', '' if i < 1 else sequence[i-1] + ' ' + past_tags[i-1])
            add_feature('2-prev-word-with-tag', '' if i < 2 else sequence[i-2] + ' ' + past_tags[i-2])
            add_feature('3-prev-word-with-tag', '' if i < 3 else sequence[i-3] + ' ' + past_tags[i-3])
            add_feature('2-prev-word', '' if i < 2 else sequence[i-2])
            add_feature('3-prev-word', '' if i < 3 else sequence[i-3])
            add_feature('next-word', '' if i >= len(sequence)-1 else sequence[i+1])
            add_feature('2-next-word', '' if i >= len(sequence)-2 else sequence[i+2])
            add_feature('3-next-word', '' if i >= len(sequence)-3 else sequence[i+3])
        elif language == "chinese":
            add_feature('bias')
            add_feature('is_first', str(i==0))
            add_feature('is_last', str(i==len(sequence)-1))
            add_feature('is_numeric', str(word.isdigit()))
            add_feature('is_alphanumeric', str(word.isalnum()))
            add_feature('4-prev-tag', '' if i < 4 else past_tags[i-4])
            add_feature('3-prev-tag', '' if i < 3 else past_tags[i-3])
            add_feature('2-prev-tag', '' if i < 2 else past_tags[i-2])
            add_feature('prev-tag', '' if i < 1 else past_tags[i-1])
            add_feature('word', word)
            add_feature('prev-word', '' if i < 1 else sequence[i-1])
            add_feature('prev-word-with-tag', '' if i < 1 else sequence[i-1] + ' ' + past_tags[i-1])
            add_feature('2-prev-word-with-tag', '' if i < 2 else sequence[i-2] + ' ' + past_tags[i-2])
            add_feature('3-prev-word-with-tag', '' if i < 3 else sequence[i-3] + ' ' + past_tags[i-3])
            add_feature('2-prev-word', '' if i < 2 else sequence[i-2])
            add_feature('3-prev-word', '' if i < 3 else sequence[i-3])
            add_feature('next-word', '' if i >= len(sequence)-1 else sequence[i+1])
            add_feature('2-next-word', '' if i >= len(sequence)-2 else sequence[i+2])
            add_feature('3-next-word', '' if i >= len(sequence)-3 else sequence[i+3])
            add_feature('prev-curr-next-word', '' if i < 1 else sequence[i-1], sequence[i], '' if i >= len(sequence)-1 else sequence[i+1])
            add_feature('prev-curr-word', '' if i < 1 else sequence[i-1], sequence[i])
            add_feature('curr-next-word', sequence[i], '' if i >= len(sequence)-1 else sequence[i+1])
            add_feature('2-prev-curr-2-next-word', '' if i < 2 else sequence[i-2], '' if i < 1 else sequence[i-1], sequence[i], '' if i >= len(sequence)-1 else sequence[i+1], '' if i >= len(sequence)-2 else sequence[i+2])
        else:
            raise Exception('Please make sure you choose either "english" or "chinese" for the language during training and/or tagging.')
        
        return features
        
        
    def train(self, train_text_path, language, freq_thresh=5,  ambiguity_thresh=0.98, it=1, lr=1.0):
        """
        Train averaged perceptron model.
        
        Parameters:
        train_text_path (string): File path of the training dataset
        language (string): Language to determine choice of feature templates
        freq_thresh (int): Frequency threshold to determine whether a word should be stored for easier non-predictive tagging (default 5)
        ambiguity_thresh (float): Ambiguity threshold to determine whether a word should be stored for easier non-predictive tagging (default 0.95)
        it (int): Number of epochs to train the averaged perceptron model for (default 1)
        lr (float): A value between 0.0 and 1.0 to determine how much to update the weights for each classification mistake (default 1.0)
        
        """
        assert train_text_path is not None
        assert type(freq_thresh) == int
        assert type(ambiguity_thresh) == float
        assert language is not None
        assert type(lr) == float
        assert lr >= 0
        assert lr <= 1

        # load training data
        with open(train_text_path) as file:
            train_data = file.readlines()
        file.close()
      
        words = []
        tags = []
        sequences = []

        # check for uncommon words
        freq = defaultdict(lambda: defaultdict(int))
        
        # get training sequences
        for i in range(len(train_data)):
            train_data[i] = train_data[i].strip('\n')
            if train_data[i] != '':
                data = train_data[i].split(' ')
                words.append(data[0])
                tags.append(data[1])
                freq[data[0]][data[1]] += 1
                if data[1] not in self.tags:
                    self.tags.add(data[1])
            else:
                sequences.append((words,tags))
                words = []
                tags = []

        # check for common and unambiguous words that can be added to tag dictionary for easier, non-predictive access
        for word, tag in freq.items():
            # get most common tag count
            most_common_tag, most_common_tag_count = max(tag.items(), key=lambda item: item[1])
            # get word count
            word_count = sum(tag.values())
            # make sure tag appears often enough and belongs to a certain tag at a rate more than ambiguity_thresh
            if float(most_common_tag_count) / word_count >= ambiguity_thresh and word_count >= freq_thresh:
                self.tagdict[word] = most_common_tag

        # training the model
        for i in range(it):
            for seq, tags in sequences:
                past_tags=[]
                for j, word in enumerate(seq):
                    # get tag prediction if it is the usual one for the word, from the easy access tag dictionary
                    y_pred = self.tagdict.get(word)
                    
                    # use perceptron prediction if it is not
                    if not y_pred:
                        # update time for each perceptron run
                        self.time += 1
                        
                        feats = self.get_features(j, word, seq, past_tags, language)
                        y_pred = self.predict(feats)

                        # if prediction is incorrect --> update weight for each feature according to perceptron rules
                        if y_pred != tags[j]:
                            for feat in feats:
                                feature_weight = self.weights.setdefault(feat, {})
                                
                                # update weight for ground truth tag
                                real_tag_weight = feature_weight.get(tags[j], 0.0)
                                self.total_weights[feat][tags[j]] += (self.time - self.update_time[feat][tags[j]]) * real_tag_weight
                                self.update_time[feat][tags[j]] = self.time
                                self.weights[feat][tags[j]] = real_tag_weight + lr * 1.0
                                
                                # update weight for y_pred tag
                                pred_tag_weight = feature_weight.get(y_pred, 0.0)
                                self.total_weights[feat][y_pred] += (self.time - self.update_time[feat][y_pred]) * pred_tag_weight
                                self.update_time[feat][y_pred] = self.time
                                self.weights[feat][y_pred] = pred_tag_weight + lr * (-1.0)
                    
                    past_tags.append(y_pred)
                    
            # shuffle sequences after each epoch for more balanced updates        
            random.shuffle(sequences)

            print("Training done for epoch " + str(i + 1) + "...")
        
        # average weights for each feature across all timesteps
        for feature, feat_tag_weights in self.weights.items():
            averaged_feat_weights = {}
            for tag, tag_weight in feat_tag_weights.items():
                # update total weights for all features and all tags
                self.total_weights[feature][tag] += (self.time - self.update_time[feature][tag]) * tag_weight
                averaged =  self.total_weights[feature][tag] / float(self.time)
                if averaged is not None:
                    averaged_feat_weights[tag] = averaged
            self.weights[feature] = averaged_feat_weights
        
    
    def predict(self, features):
        """
        Predict best tag based on features and current weights.
        
        Parameters:
        features (dict): Dictionary with feature names and feature values
        
        Returns:
        string: Most likely tag output for the feature representation passed in
        
        """
        scores = defaultdict(float)
        # get features
        for feat, feat_value in features.items():
            if feat in self.weights:
                # get feature and tag pair weight
                feat_tag_weights = self.weights[feat]
                for tag, tag_weight in feat_tag_weights.items():
                    scores[tag] += feat_value * tag_weight

        # return tag with the best score, after going through all features
        return max(self.tags, key=lambda tag: (scores[tag], tag))

        
    def tag(self, test_text_path, output_path, language, show_freq=100):
        """
        Tag test sequences.
        
        Parameters:
        test_text_path (string): File path of the test dataset
        output_path (string): File path of the output sequences with predicted tags
        language (string): Language to determine choice of feature templates
        show_freq (int): How many predictions to complete before printing progress to the console (default 100)
        
        """
        assert test_text_path is not None
        assert output_path is not None
        assert language is not None
        assert type(show_freq) == int

        # load test data
        with open(test_text_path) as file:
            test_data = file.readlines()
        file.close()

        # get test sequences
        test_seq = []
        temp_input = []
        for i in range(len(test_data)):
            test_data[i] = test_data[i].strip('\n')
            if test_data[i] != '':
                temp_input.append(test_data[i])
            else:
                test_seq.append(temp_input)
                temp_input = []

        output = []
        count = 0

        print("\n")
        for seq in test_seq:
            past_tags = []
            count += 1

            for i, word in enumerate(seq):
                # check if word exists in the easy access tag dictionary
                y_pred = self.tagdict.get(word)
                if not y_pred:
                    feats = self.get_features(i, word, seq, past_tags, language)
                    y_pred = self.predict(feats)
                past_tags.append(y_pred)
                output.append(word + ' ' + y_pred + '\n')
            
            output.append('\n')

            if count % show_freq == 0:
                print("Prediction done for " + str(count) + " test sequences...")
        
        # write model predictions as outputs to output_path
        with open(output_path, 'w') as f:
            for item in output:
                f.write(item)
        f.close()

        print('\nOutputs written to ' + str(output_path) + '!')

        
# TODO: get the averaged perceptron outputs for your test dataset
        
print("\n\n**EN Development Set**")
en_tagger = AveragedPerceptronTagger()
en_tagger.train('./dataset/en/EN/train', it=5, language='english')
en_tagger.tag('./dataset/en/EN/dev.in','./dataset/en/EN/dev.p5.out', language='english')

print("\n\n**AL Development Set**")
al_tagger = AveragedPerceptronTagger()
al_tagger.train('./dataset/al/AL/train', it=10, language='chinese')
al_tagger.tag('./dataset/al/AL/dev.in','./dataset/al/AL/dev.p5.out', language='chinese')



**EN Development Set**
Training done for epoch 1...
Training done for epoch 2...
Training done for epoch 3...
Training done for epoch 4...
Training done for epoch 5...


Prediction done for 100 test sequences...
Prediction done for 200 test sequences...
Prediction done for 300 test sequences...
Prediction done for 400 test sequences...
Prediction done for 500 test sequences...
Prediction done for 600 test sequences...
Prediction done for 700 test sequences...
Prediction done for 800 test sequences...
Prediction done for 900 test sequences...
Prediction done for 1000 test sequences...

Test results written to ./dataset/en/EN/dev.p5.out!


**AL Development Set**
Training done for epoch 1...
Training done for epoch 2...
Training done for epoch 3...
Training done for epoch 4...
Training done for epoch 5...
Training done for epoch 6...
Training done for epoch 7...
Training done for epoch 8...
Training done for epoch 9...
Training done for epoch 10...


Prediction done for 100 test sequenc

In [37]:
# Evaluation results for Part 5

print("\n\n**EN Development Set**")
!python3 evalscript/evalResult.py dataset/en/EN/dev.out dataset/en/EN/dev.p5.out
print("\n\n**AL Development Set**")
!python3 evalscript/evalResult.py dataset/al/AL/dev.out dataset/al/AL/dev.p5.out



**EN Development Set**

#Entity in gold data: 13179
#Entity in prediction: 13427

#Correct Entity : 12141
Entity  precision: 0.9042
Entity  recall: 0.9212
Entity  F: 0.9127

#Correct Sentiment : 11872
Sentiment  precision: 0.8842
Sentiment  recall: 0.9008
Sentiment  F: 0.8924


**AL Development Set**

#Entity in gold data: 8408
#Entity in prediction: 8776

#Correct Entity : 7597
Entity  precision: 0.8657
Entity  recall: 0.9035
Entity  F: 0.8842

#Correct Sentiment : 7168
Sentiment  precision: 0.8168
Sentiment  recall: 0.8525
Sentiment  F: 0.8343


In [38]:
# Test set predictions for Part 5

print("\n\n**EN Test Set**")
en_tagger.tag('./dataset/en/EN/test.in','./dataset/en/EN/test.p5.out', language='english')
print("\n\n**AL Test Set**")
al_tagger.tag('./dataset/al/AL/test.in','./dataset/al/AL/test.p5.out', language='chinese')



**EN Test Set**


Prediction done for 100 test sequences...
Prediction done for 200 test sequences...
Prediction done for 300 test sequences...
Prediction done for 400 test sequences...
Prediction done for 500 test sequences...
Prediction done for 600 test sequences...
Prediction done for 700 test sequences...
Prediction done for 800 test sequences...
Prediction done for 900 test sequences...
Prediction done for 1000 test sequences...
Prediction done for 1100 test sequences...
Prediction done for 1200 test sequences...
Prediction done for 1300 test sequences...
Prediction done for 1400 test sequences...
Prediction done for 1500 test sequences...
Prediction done for 1600 test sequences...
Prediction done for 1700 test sequences...
Prediction done for 1800 test sequences...
Prediction done for 1900 test sequences...
Prediction done for 2000 test sequences...
Prediction done for 2100 test sequences...

Test results written to ./dataset/en/EN/test.p5.out!


**AL Test Set**


Prediction d